In [ ]:
import torch , math
torch.manual_seed(42)

lr = 0.01
q = torch.randn(1 , 8 , 8 , 64) #(batch , numHEAD , seq_len , head_dim)
k = torch.randn(1 , 8 , 8 , 64)
v = torch.randn(1 , 8 , 8 , 64)

q.requires_grad_(True)
k.requires_grad_(True)
v.requires_grad_(True)


y_true = torch.randint(0, 64, (1, 8))

# y = wx + b  , where x is the data
for epoch in range(50):
    # z = torch.matmul(dummy , weights) + bias
    scores = torch.matmul(q , k.transpose(-2 , -1)) / math.sqrt(k.shape[-1])
    
    max_ = torch.max(scores , dim = -1 , keepdim = True)[0]
    exp_z = torch.exp(scores - max_) 
    sum_ = torch.sum(exp_z, dim=-1, keepdim=True)
 
    P = exp_z / sum_

    S = torch.matmul(P , v)

    S_mean = S.mean(dim=1)  

    log_probs = torch.nn.functional.log_softmax(S_mean, dim=-1)
    loss = -log_probs[torch.arange(1).unsqueeze(1), torch.arange(8), y_true].mean()

    ds = torch.nn.functional.softmax(S_mean , dim = 1)
    ds[torch.arange(1).unsqueeze(1) , torch.arange(8) , y_true] -= 1
    ds = ds / (1 * 8)

    num_heads = 8
    ds_expanded = ds.unsqueeze(1).expand(-1, num_heads, -1, -1) / num_heads

    dv = torch.matmul(P.transpose(-2, -1), ds_expanded) # [1, 8, 8, 64]
    dp = torch.matmul(ds_expanded, v.transpose(-2, -1)) # [1, 8, 8, 8]

    d_scores = P * (dp - torch.sum(P * dp, dim=-1, keepdim=True))

    scale = 1.0 / math.sqrt(k.shape[-1])

    dq = torch.matmul(d_scores * scale , k)

    dk = torch.matmul(d_scores.transpose(-2 , -1) * scale , q)
    
    with torch.no_grad():
        q -= lr * dq
        k -= lr * dk
        v -= lr * dv

    print(f"loss for epoch {epoch} : {loss.item()}")


    
    

    


In [29]:
import torch , math
torch.manual_seed(42)

input_dim = 512
output_dim = 64
lr = 0.1

x = torch.randn(1 , 8 , 8 , 512) # (1 , 8 , 8 , 512)


weights = torch.randn(input_dim , output_dim , requires_grad = False).unsqueeze(0) #(1 , 512 , 64)
bias = torch.zeros(output_dim)

alpha = torch.randn(1 , 1 , 1 , 64)
b = torch.zeros(64)


y_true = torch.randint(0, 64, (1, 8))

for epoch in range(1 , 200):
    z = torch.matmul(x , weights) + bias #(1 , 8 , 8 , 64)
    
    mean = z.mean(dim = -1 , keepdim = True) #[1 , 8 , 8 , 1]
    std = z.var(dim = -1 , keepdim = True , unbiased = False) #[1 , 8 , 8 , 1]
    
    xCAP = (z - mean) / (torch.sqrt(std + 1e-9)) 
    y = xCAP * alpha + b

    maxx = torch.max(y , dim = -1 , keepdim = True)[0]
    exp = torch.exp(y - maxx)
    summ = torch.sum(exp , dim = -1 , keepdim = True)

    S = (exp / summ) / z.size(0)
    S_mean = S.mean(dim=1)  

    log_probs = torch.nn.functional.log_softmax(S_mean, dim=-1)
    loss = -log_probs[... , y_true].mean()

    ds = torch.nn.functional.softmax(S_mean , dim = 1) # (1 , 8 , 64)
    ds[... , y_true] -= 1
    ds = ds / (1 * 8)
    
    dy = ds.unsqueeze(1).expand(-1, 8, -1, -1) / 8  # [1, 8, 8, 64]

    n = z.shape[-1]
    # dx , dvar , dmean  grads wth direct , variance , mean

    d_alpha = (dy * xCAP).sum(dim=(0,1,2), keepdim=True)  # [1, 1, 1, 64]
    d_b = dy.sum(dim=(0,1,2), keepdim=True)

    dxCAP = dy * alpha

    dz_direct = dxCAP / std

    #via variance 
    dz_sum = (dxCAP * xCAP).sum(dim = -1 , keepdim = True)
    dz_var = -(1.0 / std) * (xCAP) * (dz_sum / n)

    # via mean
    dz_sum_ = dxCAP.sum(dim = -1 , keepdim = True)
    dz_mean = -(1.0 / std) * (dz_sum_ / n)  

    dz = dz_direct + dz_var + dz_mean

    d_weights = torch.matmul(x.transpose(-2, -1), dz).sum(dim=1)  # [512, 64]
    d_bias = dz.sum(dim=(0,1,2))

    d_b_squeezed = d_b.squeeze()

    with torch.no_grad():
        weights -= lr * d_weights
        bias -= lr * d_bias
        alpha -= lr * d_alpha
        b -= lr * d_b_squeezed
    
    print(f"loss over epoch {epoch} : {loss.item()}")
    

loss over epoch 1 : 4.164918899536133
loss over epoch 2 : 4.163978576660156
loss over epoch 3 : 4.162956237792969
loss over epoch 4 : 4.16184663772583
loss over epoch 5 : 4.160642623901367
loss over epoch 6 : 4.159340858459473
loss over epoch 7 : 4.157934665679932
loss over epoch 8 : 4.156418800354004
loss over epoch 9 : 4.154788970947266
loss over epoch 10 : 4.153040885925293
loss over epoch 11 : 4.151169776916504
loss over epoch 12 : 4.149171829223633
loss over epoch 13 : 4.1470465660095215
loss over epoch 14 : 4.1447906494140625
loss over epoch 15 : 4.142405033111572
loss over epoch 16 : 4.139889717102051
loss over epoch 17 : 4.137246131896973
loss over epoch 18 : 4.134479522705078
loss over epoch 19 : 4.131594181060791
loss over epoch 20 : 4.128596305847168
loss over epoch 21 : 4.125494003295898
loss over epoch 22 : 4.122298240661621
loss over epoch 23 : 4.119019508361816
loss over epoch 24 : 4.115668773651123
loss over epoch 25 : 4.112261772155762
loss over epoch 26 : 4.1088113784